In [31]:
# Libraries required along the code
import numpy as np
np.set_printoptions(threshold = np.inf)
#from UART_RX import *

In [32]:
#===========================================================================
# MODULE Definition
#===========================================================================
# Function describing UART serial communication protocol for reception
def UART_RX (UART_RX_CLOCK_50,UART_RX_RESET_InHigh,UART_RX_rx_InLow,
             CLOCK_PER_BIT,DATAWIDTH_BUS,STATE_SIZE):
    
    #===========================================================================
    # PARAMETER Declarations
    #===========================================================================
    #//////////// STATES ////////////
    State_IDLE = np.uint8(0)
    State_WAIT_HALF = np.uint8(1)
    State_WAIT_FULL_INIT = np.uint8(2)
    State_WAIT_FULL_END = np.uint8(3)
    State_WAIT_HIGH = np.uint8(4)
    #//////////// SIZES ////////////
    # Ceiling of log base 2 to get the number of bits needed to store a value
    # CLOCK_PER_BIT = 50MHz/256000Bauds
    COUNTER_SIZE = int(np.log2(CLOCK_PER_BIT)) # 8 bits
    
    #===========================================================================
    # PORT Declarations
    #===========================================================================
    #//////////// OUTPUTS ////////////
    # Default values
    UART_RX_newData_Out = np.uint8(1)
    UART_RX_data_Out = np.zeros(DATAWIDTH_BUS,dtype = 'uint8')

    #===========================================================================
    # REG/WIRE Declarations
    #===========================================================================
    # Registers (Q)
    global NewData_Register,Data_Register,Rx_Register,State_Register
    global Counter_Register,BitCounter_Register
    
    #===========================================================================
    # STRUCTURAL Coding
    #===========================================================================
    # INPUT LOGIC: Combinational
    # Signals (D)
    Rx_Signal = UART_RX_rx_InLow
    State_Signal = State_Register
    Counter_Signal = Counter_Register
    BitCounter_Signal = BitCounter_Register

    if State_Register == State_IDLE:
        Counter_Signal = 0
        BitCounter_Signal = 0
        if Rx_Register == np.uint8(0):
            State_Signal = State_WAIT_HALF
        else:
            State_Signal = State_IDLE
            
    elif State_Register == State_WAIT_HALF:
        Counter_Signal = Counter_Register + 1
        # The counter is compared with half of the CLOCK_PER_BIT
        if Counter_Register == (CLOCK_PER_BIT >> np.uint8(1)):
            State_Signal = State_WAIT_FULL_INIT
            Counter_Signal = 0
        else:
            State_Signal = State_WAIT_HALF

    elif State_Register == State_WAIT_FULL_INIT:
        Counter_Signal = Counter_Register + 1
        if Counter_Register == CLOCK_PER_BIT-1:
            State_Signal = State_WAIT_FULL_END
            Counter_Signal = 0
        else:
            State_Signal = State_WAIT_FULL_INIT

    elif State_Register == State_WAIT_FULL_END:     
        BitCounter_Signal = BitCounter_Register + 1
        if BitCounter_Register == DATAWIDTH_BUS-1:
            State_Signal = State_WAIT_HIGH
            Counter_Signal = 0
        else:
            State_Signal = State_WAIT_FULL_INIT

    elif State_Register == State_WAIT_HIGH:
        Counter_Signal = Counter_Register + 1
        if Rx_Register == np.uint8(1):
            State_Signal = State_IDLE
            Counter_Signal = 0
        else:
            State_Signal = State_WAIT_HIGH

    else:
        State_Signal = State_IDLE
    
    #===========================================================================
    # OUTPUTS
    #===========================================================================
    # OUTPUT LOGIC: Combinational
    # Signals (D)
    NewData_Signal = NewData_Register
    
    if State_Register == State_IDLE:
        NewData_Signal = np.uint8(0)
        Data_Signal = Data_Register

    elif State_Register == State_WAIT_HALF:
        NewData_Signal = np.uint8(0)
        Data_Signal = Data_Register        

    elif State_Register == State_WAIT_FULL_INIT:
        NewData_Signal = np.uint8(0)
        Data_Signal = Data_Register

    elif State_Register == State_WAIT_FULL_END:
        NewData_Signal = np.uint8(1)
        Data_Signal = np.array([Data_Register[i] for i in range(1,8)])
        Data_Signal = np.insert(Data_Signal,7,Rx_Register,axis = 0)
        
    elif State_Register == State_WAIT_HIGH:
        NewData_Signal = np.uint8(0)
        Data_Signal = Data_Register        

    else:
        NewData_Signal = np.uint8(0)
        Data_Signal = Data_Register        
        
    # STATE REGISTER : Sequential
    if UART_RX_CLOCK_50:
        # Updating global registers
        if UART_RX_RESET_InHigh:
            NewData_Register = np.uint8(0)
            Data_Register = np.zeros(DATAWIDTH_BUS,dtype = 'uint8')
            Rx_Register = np.uint8(1)
            State_Register = State_IDLE
            Counter_Register = 0
            BitCounter_Register = 0
        else:
            NewData_Register = NewData_Signal
            Data_Register = Data_Signal
            Rx_Register = Rx_Signal
            State_Register = State_Signal
            Counter_Register = Counter_Signal
            BitCounter_Register = BitCounter_Signal

    # OUTPUT ASSIGNMENTS
    UART_RX_newData_Out = NewData_Register
    UART_RX_data_Out = Data_Register
    
    return UART_RX_newData_Out,UART_RX_data_Out

In [33]:
#===========================================================================
# MODULE Definition
#===========================================================================
# Function in charge of the input and output signals (I/O)
def BB_SYSTEM (BB_SYSTEM_CLOCK_50,BB_SYSTEM_RESET_InHigh,BB_SYSTEM_rx_InLow):
    #===========================================================================
    # PARAMETER Declarations
    #===========================================================================
    # Ratio between the internal frequency and the baud rate
    # CLOCK_PER_BIT = 50MHz/256000Bauds
    FREQUENCY = 50e6
    BAUD_RATE = 115200
    CLOCK_PER_BIT = int(FREQUENCY/BAUD_RATE)
    # Data width of the imput bus
    DATAWIDTH_BUS = 8
    # Size for the states needed into the protocol
    STATE_SIZE = 3

    #===========================================================================
    # PORT Declarations
    #===========================================================================
    #//////////// OUTPUTS ////////////
    # Default values
    BB_SYSTEM_newData_Out = np.uint8(0)
    BB_SYSTEM_data_Out = np.zeros(DATAWIDTH_BUS,dtype = 'uint8')
    
    #===========================================================================
    # REG/WIRE Declarations
    #===========================================================================
    
    #===========================================================================
    # STRUCTURAL Coding
    #===========================================================================
    # Port map - connection between master ports and signals/registers  
    BB_SYSTEM_newData_Out,BB_SYSTEM_data_Out = UART_RX (BB_SYSTEM_CLOCK_50,BB_SYSTEM_RESET_InHigh,BB_SYSTEM_rx_InLow,
                                                        CLOCK_PER_BIT,DATAWIDTH_BUS,STATE_SIZE)
    
    # OUTPUT ASSIGNMENTS
    return BB_SYSTEM_newData_Out,BB_SYSTEM_data_Out

In [42]:
#===========================================================================
# SIMULATION Coding
#===========================================================================
# TB_SYSTEM
# A text file is read and opened with the output of the UART transmitter module
file = "tx_Out.txt"
rx_InLow = np.loadtxt(file,dtype = 'uint8')

# The variables to treat the file are created
nBits = 8

# Variable to emulate the 50MHz clock        
CLOCK_50 = 0
        
# Parameters to describe the size of the arrays
# CLOCK_PER_BIT = 50MHz/256000Bauds
FREQUENCY = 50e6
BAUD_RATE = 115200
CLOCK_PER_BIT = int(FREQUENCY/BAUD_RATE)

# The test vectors to use for the communication protocol are created
nSim = 4
resetV = np.zeros(nSim,dtype = 'uint8')
nRows = int(np.size(rx_InLow)/(((nBits + 2)*CLOCK_PER_BIT + nBits + 2)))

# Arrays composed of protocol outputs in each clock cycle
newData_OutV = np.zeros(np.size(rx_InLow),dtype = 'uint8')
data_OutV = np.zeros((nBits,nRows*nBits),dtype = 'uint8')

# The communication protocol is tested for the CLOCK_PER_BIT given value 
r = 0

while r < nSim:
    s = 0
    t = 0
    
    # The communication protocol registers are defined on its initial values
    NewData_Register = np.uint8(0)
    Data_Register = np.zeros(nBits,dtype = 'uint8')
    Rx_Register = np.uint8(1)
    State_Register = np.uint8(0)
    Counter_Register = 0
    BitCounter_Register = 0

    while s < np.size(rx_InLow):
        CLOCK_50 = np.uint8(0) if CLOCK_50 else np.uint8(1)
        newData_Out,data_Out = BB_SYSTEM (CLOCK_50,resetV[r],rx_InLow[s])
        if CLOCK_50:
            newData_OutV[s] = newData_Out
            if newData_Out == np.uint8(1):
                data_OutV[:,t] = data_Out
                #print('Out: ',data_OutV[:,t])
                t = t + 1
                
            s = s + 1
    r = r + 1

# Out to the data bus module
np.savetxt('data_Out.txt',data_OutV,fmt = '%u')
print(data_OutV)

[[0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 0
  0 1 1 0 0 1 1 0 1 0 0 0 0 1 1 0 1 0 1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 1 1 0
  0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 0 0
  0 1 1 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0
  1 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 0 0
  1 1 0 0 1 1 0 1 0 0 0 0 1 1 0 1 0 1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0
  1 1 1 0 0 0 0 0 1 1 1 0 0 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 0 0 0
  1 1 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 1
  1 0 1 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 0 0 1
  1 0 0 1 1 0 1 0 0 0 0 1 1 0 1 0 1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0 1
  1 1 0 0 0 0 0 1 1 1 0 0 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1
  1 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 1 1
  0 1 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 1